## Подготовка

In [24]:
# Импорт библиотек

import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
m = WordNetLemmatizer() 
import re
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords as stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [4]:
df = pd.read_csv('D:\Start\Data scientist 2.0\Projects\Project_Texts/toxic_comments.csv')

In [5]:
display(df.head(5))
df.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


### Очистка

In [6]:
# Лемматизация и токенизация
def text_tokenize(text):
    tokenized = nltk.word_tokenize(text)
    joined = ' '.join(tokenized)
    text_only = re.sub(r'[^a-zA-Z]', ' ', joined)
    final = ' '.join(text_only.split())
    x =  "".join(m.lemmatize(final))
    return x

In [8]:
df['text_final'] = df['text'].apply(text_tokenize)

In [9]:
df.head(5)

,text,toxic,text_final
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I ca n t make any real suggestions on imp...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


Создадим признаки и целевой признак

In [10]:
features = df['text_final']
target = df['toxic']

In [11]:
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            random_state=12345, test_size=0.2, stratify=target)
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(127656,)
(127656,)
(31915,)
(31915,)


Создадим корпус

In [12]:
corpus_train = features_train.values.astype('U')
corpus_test = features_test.values.astype('U')

In [13]:
print(corpus_train.shape)
print(corpus_test.shape)

(127656,)
(31915,)


Создадим стоп-слова, чтобы избавиться от предлогов, союзов и прочего шума.

In [14]:
stop_words = set(stopwords.words('english'))

TF-IDF

In [15]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words) 

In [18]:
features_train_tf = count_tf_idf.fit_transform(corpus_train) 
features_test_tf = count_tf_idf.transform(corpus_test)

In [22]:
display(features_test_tf.shape)
features_train_tf.shape

(31915, 147731)

(127656, 147731)

Теперь данные готовы к обучению

## Обучение

### Случайный лес

Добавим баланс классов к случайному лесу

In [27]:
%%time
model_forest = RandomForestClassifier(random_state=12345, class_weight = 'balanced')
model_forest.fit(features_train_tf, target_train)
predictions_forest = model_forest.predict(features_test_tf)
    
forest_f1 = f1_score(target_test, predictions_forest)

Wall time: 15min 3s


In [29]:
forest_f1

0.6479044340959709

### Логистическая регрессия

In [28]:
%%time
model_reg = LogisticRegression(random_state=12345, class_weight='balanced')
model_reg.fit(features_train_tf, target_train)
predictions_reg = model_reg.predict(features_test_tf)
    
reg_f1 = f1_score(target_test, predictions_reg)

Wall time: 6.86 s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
reg_f1

0.7603533959138598

## Выводы

В данном исследовании была проведена лемматизация, токенизация и векторизация комментариев пользователей. По результатам метрики F1 лучшей моделью оказалась логистическая регресси, её результатом стал 0.76 на тестовой выборке.. 